In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = 'cuda'

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from pytorchtools import EarlyStopping
assert(torch.cuda.is_available())

## Data Preprocessing

In [3]:
# import logging
# def logger(content):
#     logging.getLogger('matplotlib.font_manager').disabled = True
#     log_format = '[%(asctime)s] %(message)s'
#     date_format = '%Y-%m-%d %H:%M:%S'
#     logging.basicConfig(level = logging.DEBUG, format = log_format, datefmt = date_format)
#     logging.info(content)

In [ ]:
# # merge news to one document
# news_set = set()
# news = []
# with open('/data/Recommend/MIND/MINDsmall_train/news.tsv', 'r') as f:
#     for line in f:
#         data = line.split('\t')
#         news_id = data[0]
#         if news_id not in news_set:
#             news.append(line)
#             news_set.add(news_id)
# with open('/data/Recommend/MIND/MINDsmall_dev/news.tsv') as f:
#     for line in f:
#         data = line.split('\t')
#         news_id = data[0]
#         if news_id not in news_set:
#             news.append(line)
#             news_set.add(news_id)
# # with open('/data/Recommend/MIND/MINDlarge_test/news.tsv') as f:
# #     for line in f:
# #         data = line.split('\t')
# #         news_id = data[0]
# #         if news_id not in news_set:
# #             news.append(line)
# #             news_set.add(news_id)

# # with open('/data/Recommend/MIND/small_news.tsv', 'w') as f:
# #     f.writelines(news)

# print(len(news))

In [3]:
def load_news(path):
    news_dict = {} # index -> news
    news_list = [] # index -> news
    newsid_dict = {} # newsid -> index
    word_dict = {'<PAD>': 0, '<OOV>': 1}
    cate_dict = {'<PAD>': 0, '<OOV>': 1}
    with open(path, 'r') as f:
        for line in f.readlines():
            news_id, category, subcategory, title, abstract, \
                url, title_entities, abstract_entities = line.strip().split('\t')
            title = title.lower().replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('\'', '').replace('"', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').split(' ')
            abstract = abstract.lower().replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('\'', '').replace('"', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').split(' ')
            for word in title + abstract:
                if word not in word_dict:
                    word_dict[word] = len(word_dict)
            if category not in cate_dict:
                cate_dict[category] = len(cate_dict)
            if subcategory not in cate_dict:
                cate_dict[subcategory] = len(cate_dict)
            if news_id not in newsid_dict:
                newsid_dict[news_id] = len(newsid_dict)
                news_list.append([category, subcategory, title, abstract])
    print(len(news_list))
    return news_list, newsid_dict, word_dict, cate_dict

In [4]:
max_title = 30
max_body = 100
def map_news_input(news_list, word_dict, cate_dict):
    n_news = len(news_list)
    titles = np.zeros((n_news, max_title), dtype = 'int32')
    bodys = np.zeros((n_news, max_body), dtype = 'int32')
    cates = np.zeros((n_news,1), dtype = 'int32')
    subcates = np.zeros((n_news,1), dtype = 'int32')
    for i in range(n_news):
        category, subcategory, title, abstract = news_list[i]
        titles[i, :len(title)] = [word_dict[word] for word in title[:max_title]]
        bodys[i, :len(abstract)] = [word_dict[word] for word in abstract[:max_body]]
        cates[i] = cate_dict[category]
        subcates[i] = cate_dict[subcategory]
    news_info = np.concatenate((titles, bodys, cates, subcates), axis = 1)
    print(news_info.shape)
    return news_info # index -> news_info

In [5]:
'''
news_list: original news
news_info: mapped news(word ids)
'''
news_list, newsid_dict, word_dict, cate_dict = load_news('/data/Recommend/MIND/small_news.tsv')
news_info = map_news_input(news_list, word_dict, cate_dict)

65238
(65238, 132)


In [6]:
def load_glove(word_to_ix, dim = 100):
    if dim == 100:
        path = '/data/pretrained/Glove/glove.6B.100d.txt'
    elif dim == 300:
        path = '/data/pretrained/Glove/glove.840B.300d.txt'
    word_emb = []
    word_emb = np.zeros((len(word_to_ix), dim), dtype = float)
    with open(path, 'r') as f:
        for line in f:
            data = line.strip().split(' ') # [word emb1 emb2 ... emb n]
            word = data[0]
            if word in word_to_ix:
                word_emb[word_to_ix[word]] = [float(i) for i in data[1:]]
    print(word_emb.shape)
    return torch.tensor(word_emb, dtype = torch.float)

In [7]:
word_emb = load_glove(word_dict, 300)
cate_emb = load_glove(cate_dict, 100)

(80416, 300)
(282, 100)


In [8]:
def load_train_impression(path, newsid_dict): # train&dev
    logs = []
    with open(path, 'r') as f:
        for line in f:
            imp_id, user_id, time, history, impression = line.strip().split('\t')
            if history:
                history = [newsid_dict[news_id] for news_id in history.split(' ')]
            else:
                history = []
            positive = []
            negative = []
            for item in impression.split(' '):
                news_id, num = item.split('-')
                if num == '1':
                    positive.append(newsid_dict[news_id])
                else:
                    negative.append(newsid_dict[news_id])
            logs.append([history, positive, negative]) # indexs
    return logs

In [9]:
max_history = 50
def map_user(logs): # index -> history, 用 index 代表 user_id, train&dev
    n_user = len(logs)
    user_hist = np.zeros((n_user, max_history), dtype = 'int32') # index -> history
    for i in range(n_user):
        history, positive, negative = logs[i]
        n_hist = len(history)
        if n_hist == 0:
            continue
        user_hist[i, -n_hist:] = history[-max_history:]
    return user_hist         

In [10]:
neg_ratio = 4
def neg_sample(negative):
    if len(negative) < neg_ratio:
        return random.sample(negative * (neg_ratio // len(negative) + 1), neg_ratio)
    else:
        return random.sample(negative, neg_ratio)

def get_train_input(logs): # 和 map_user 使用同一个 log
    all_pos = [] # 每个 sample 的 pos
    all_neg = []
    user_id = [] # 每个 sample 的 user，用 index 表示，和 map_user 的结果对应
    for i in range(len(logs)):
        history, positive, negative = logs[i]
        for pos in positive:
            all_pos.append(pos)
            all_neg.append(neg_sample(negative))
            user_id.append(i)
    n_imps = len(all_pos)
    imps = np.zeros((n_imps, 1 + neg_ratio), dtype = 'int32')
    for i in range(len(all_pos)):
        imps[i, 0] = all_pos[i]
        imps[i, 1:] = all_neg[i]
    user_id = np.array(user_id, dtype = 'int32')
    labels = np.zeros((n_imps, 1 + neg_ratio), dtype = 'int32')
    labels[:, 0] = 1
    print(n_imps)
    return imps, user_id, labels

def get_dev_input(logs): # 和 map_user 使用同一个 log
    imps = []
    labels = []
    user_id = np.zeros((len(logs)), dtype = 'int32') # 每个 sample 的 user index，和 map_user 的结果对应
    for i in range(len(logs)):
        history, positive, negative = logs[i]
        imps.append(np.array(positive + negative, dtype = 'int32'))
        labels.append([1] * len(positive) + [0] * len(negative))
        user_id[i] = i
    print(len(logs))
    return imps, user_id, labels

In [11]:
# # merge entity embedding to one document
# ent_set = set()
# ents = []
# with open('/data/Recommend/MIND/MINDsmall_train/entity_embedding.vec', 'r') as f:
#     for line in f:
#         ent_id = line.split('\t')[0]
#         if ent_id not in ent_set:
#             ents.append(line)
#             ent_set.add(ent_id)
# with open('/data/Recommend/MIND/MINDsmall_dev/entity_embedding.vec') as f:
#     for line in f:
#         ent_id = line.split('\t')[0]
#         if ent_id not in ent_set:
#             ents.append(line)
#             ent_set.add(ent_id)
# # with open('/data/Recommend/MIND/MINDlarge_test/entity_embedding.vec') as f:
# #     for line in f:
# #         ent_id = line.split('\t')[0]
# #         if ent_id not in ent_set:
# #             ents.append(line)
# #             ent_set.add(ent_id)

# with open('/data/Recommend/MIND/small_entity_embedding.vec', 'w') as f:
#     f.writelines(ents)

# print(len(ents))

In [11]:
def load_ent_emb(path):
    ent_emb = []
    ent_dict = {'<PAD>': 0, '<OOV>': 1}
    with open(path, 'r') as f:
        for line in f:
            data = line.strip().split('\t')
            ent_id = data[0]
            ent_dict[ent_id] = len(ent_dict)
            ent_emb.append([float(i) for i in data[1:]])
    ent_emb.insert(0, [0.] * len(ent_emb[0]))
    ent_emb.insert(0, [0.] * len(ent_emb[0]))
    ent_emb = torch.tensor(ent_emb, dtype = torch.float)
    print(ent_emb.shape)
    return ent_emb, ent_dict

In [12]:
max_ents = 5
def load_news_ent(path, ent_dict):
    n_news = len(news_list)
    news_ents = np.zeros((n_news, max_ents), dtype = 'int32')
    i = 0
    with open(path, 'r') as f:
        for line in f.readlines():
            data = line.strip().split('\t')
            ents = [ent['WikidataId'] for ent in json.loads(data[6])] + [ent['WikidataId'] for ent in json.loads(data[7])]
            news_ents[i, :len(ents)] = [ent_dict[ent] if ent in ent_dict else ent_dict['<OOV>'] for ent in ents[:max_ents]]
            i += 1
    print(len(news_ents))
    return news_ents # index -> ent_index

In [13]:
ent_emb, ent_dict = load_ent_emb('/data/Recommend/MIND/small_entity_embedding.vec')
news_ents = load_news_ent('/data/Recommend/MIND/small_news.tsv', ent_dict)

torch.Size([31453, 100])
65238


In [11]:
class TrainDataset(Dataset):
    def __init__(self, imp_datas, imp_users, imp_labels, news_info, user_clicks, batch_size, news_ents = None):
        self.imp_datas = imp_datas # (n_imps, 1 + k)
        self.imp_users = imp_users
        self.imp_labels = imp_labels
        self.news = news_info
        self.user_clicks = user_clicks
        self.batch_size = batch_size
        self.news_ents = news_ents
        
        self.n_data = imp_datas.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.n_data / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, self.n_data)
        
        data_id = self.imp_datas[start: end] # (n_batch, 1 + k)
        data_news = self.news[data_id] # (n_batch, 1 + k, news_len)
        user_id = self.imp_users[start: end] # (n_batch)
        user_news_id = self.user_clicks[user_id] # (n_batch, n_hist)
        user_news = self.news[user_news_id] # (n_batch, n_hist, news_len)
        labels = self.imp_labels[start: end] # (n_batch, 1 + k)
        
        if self.news_ents is not None:
            samp_ents = self.news_ents[data_id]
            user_ents = self.news_ents[user_news_id]
            return data_news, user_news, labels, samp_ents, user_ents
        
        return data_news, user_news, labels
    
class DevDataset(Dataset): # data 和 label 是 list，每条数据不同长度
    def __init__(self, imp_datas, imp_users, imp_labels, news_info, user_clicks, batch_size):
        self.imp_datas = imp_datas # [imp1, imp2, ..., impn]
        self.imp_users = imp_users # (n_imps)
        self.imp_labels = imp_labels
        self.news = news_info
        self.user_clicks = user_clicks
        self.batch_size = batch_size
        
        self.n_data = len(imp_datas)
        
    def __len__(self):
        return int(np.ceil(self.n_data / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, self.n_data)
        
        data_ids = []
        data_news = [] # [(n_imp, news_len)]
        labels = [] # [(n_imp)]
        for i in range(start, end):
            data_id = self.imp_datas[i] # (n_imp)
            data_ids.append(data_id)
            # data_news.append(self.news[data_id]) # (n_imp, news_len)
            labels.append(self.imp_labels[i]) # (n_imp)
        user_id = self.imp_users[start: end] # (n_batch)
        user_news_id = self.user_clicks[user_id] # (n_batch, n_hist)
        # user_news = self.news[user_news_id] # (n_batch, n_hist, news_len)
        
        #return data_news, user_news, labels
        return data_ids, user_news_id, labels

In [12]:
n_batch = 16
train_logs = load_train_impression('/data/Recommend/MIND/MINDsmall_train/behaviors.tsv', newsid_dict)
train_user_hist = map_user(train_logs)
train_datas, train_users, train_labels = get_train_input(train_logs)
train_dataset = TrainDataset(train_datas, train_users, train_labels, news_info, train_user_hist, n_batch)

dev_logs = load_train_impression('/data/Recommend/MIND/MINDsmall_dev/behaviors.tsv', newsid_dict)
dev_user_hist = map_user(dev_logs)
dev_datas, dev_users, dev_labels = get_dev_input(dev_logs)
dev_dataset = DevDataset(dev_datas, dev_users, dev_labels, news_info, dev_user_hist, 64)

valid_datas, valid_users, valid_labels = get_train_input(dev_logs) # 用 train 的方法构造 dev_set
valid_dataset = TrainDataset(valid_datas, valid_users, valid_labels, news_info, dev_user_hist, n_batch)

236344
73152
111383


In [20]:
def encode_all_news(news_info, news_encoder):
    n_news = len(news_info)
    news_rep = []
    n_batch = 32
    for i in range((len(news_info) + n_batch - 1) // n_batch):
        batch_news = torch.tensor(news_info[i * n_batch: (i + 1) * n_batch], dtype = torch.long, device = 'cuda')
        batch_rep = news_encoder(batch_news).detach().cpu().numpy()
        news_rep.append(batch_rep)
    news_rep = np.concatenate(news_rep, axis = 0)
    return news_rep # (n_news, n_title, n_emb)

def encode_all_user(user_ids, user_hist, user_encoder, news_rep):
    user_rep = []
    with torch.no_grad():
        for _, batch in enumerate(dev_dataset):
            if len(batch[0]) == 0:
                break
            user_hist_rep = torch.tensor(news_rep[batch[1]], device = 'cuda') # (n_batch, n_hist)
            user = model.user_encoder(user_hist_rep).detach().cpu().numpy() # (n_batch, emb_dim)
            user_rep.append(user)
    # user_rep = np.concatenate(user_rep, axis = 0)
    return user_rep # [user_rep, ...]

In [21]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

## Training & evaluating

In [22]:
# train with valid
def train(model, train_dataset, valid_dataset = None, epochs = 4):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_losses = []
        valid_losses = []
        model.train()
        for _, batch in tqdm(enumerate(train_dataset)):
            if batch[0].shape[0] == 0:
                break
            # torch.Size([16, 5, 30]) torch.Size([16, 50, 30]) torch.Size([16])
            sample = torch.tensor(batch[0], dtype = torch.long, device = device)
            history = torch.tensor(batch[1], dtype = torch.long, device = device)
            correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
            optimizer.zero_grad()
            output = model(history, sample)
            loss = entrophy(output, correct)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()

        if valid_dataset is not None:
            model.eval()
            with torch.no_grad():
                for _, batch in enumerate(valid_dataset):
                    if batch[0].shape[0] == 0:
                        break
                    sample = torch.tensor(batch[0], dtype = torch.long, device = device)
                    history = torch.tensor(batch[1], dtype = torch.long, device = device)
                    correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
                    output = model(sample, history)
                    loss = entrophy(output, correct)
                    valid_losses.append(loss.item())
                print('[epoch {:d}] train_loss: {:.4f} valid_loss: {:.4f}'.format(epoch + 1, np.average(train_losses), np.average(valid_losses)))
        else:
            print('[epoch {:d}] train_loss: {:.4f}'.format(epoch + 1, np.average(train_losses)))

In [23]:
def evaluate(model, dev_dataset, news_info, dev_users, dev_user_hist):
    news_rep = encode_all_news(news_info, model.news_encoder) # (65238, 400)
    user_rep = encode_all_user(dev_users, dev_user_hist, model.user_encoder, news_rep)
    
    model.eval()
    with torch.no_grad():
        auc_scores = []
        mrr_scores = []
        ndcg5_scores = []
        ndcg10_scores = []
        for i, batch in tqdm(enumerate(dev_dataset)):
            if len(batch[0]) == 0:
                break
            user = user_rep[i]
            for j in range(len(batch[0])):
                sample = news_rep[batch[0][j]] # (n_imp, emb_dim)
                positive = batch[2][j] # (1, n_imp)

                score = np.matmul(sample, user[j]) # (1, n_imp)
                predict = np.exp(score) / np.sum(np.exp(score))

                auc_scores.append(roc_auc_score(positive, predict))
                mrr_scores.append(mrr_score(positive, predict))
                ndcg5_scores.append(ndcg_score(positive, predict, k = 5))
                ndcg10_scores.append(ndcg_score(positive, predict, k = 10))
    print('[Test] AUC: {:4f}, MRR: {:4f}, nDCG5:{:4f}, nDCG10: {:4f}'.format(
        np.mean(auc_scores), np.mean(mrr_scores), np.mean(ndcg5_scores), np.mean(ndcg10_scores)
    ))

## NAML

In [17]:
class AttentionPooling(nn.Module):
    def __init__(self, emb_dim, query_dim):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, query_dim)
        self.fc2 = nn.Linear(query_dim, 1)
        
    def forward(self, x, mask = None):
        '''
        (n_batch, n_seq, emb_dim) -> (n_batch, emb_dim)
        a = q^T tanh(V * k + v)
        alpha = softmax(a)
        '''
        a = self.fc2(torch.tanh(self.fc1(x))) # (n_batch, n_seq, 1)
        if mask is not None:
            a = a.masked_fill(mask.unsqueeze(-1) == 0, -1e9)
        alpha = F.softmax(a, dim = -2) # (n_batch, n_seq, 1)
        r = torch.matmul(alpha.transpose(-2, -1), x).squeeze(-2) # (n_batch, emb_dim)
        return r

In [18]:
class TextEncoder(nn.Module):
    def __init__(self, word_embedding, word_emb_dim, 
                 filter_num, window_size, query_dim, dropout, use_relu = False
                ):
        super().__init__()
        self.use_relu = use_relu
        self.word_embedding = word_embedding
        self.cnn = nn.Conv1d(word_emb_dim, filter_num, window_size, padding = window_size // 2)
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)
        self.attn = AttentionPooling(filter_num, query_dim)
        
    def forward(self, x, mask = None):
        x_emb = self.word_embedding(x) # (n_batch, n_seq, emb_dim)
        x_emb = self.drop1(x_emb)
        x_rep = self.cnn(x_emb.transpose(2, 1)).transpose(2, 1) # (n_batch, n_seq, emb_dim)
        if self.use_relu:
            x_rep = F.relu(x_rep)
        x_rep = self.drop2(x_rep)
        x_rep = self.attn(x_rep, mask) # (n_batch, emb_dim)
        return x_rep

class CateEncoder(nn.Module):
    def __init__(self, cate_embedding, cate_emb_dim, out_dim, dropout = 0.2):
        super().__init__()
        self.cate_embedding = cate_embedding
        self.fc = nn.Linear(cate_emb_dim, out_dim)
        self.drop = nn.Dropout(dropout)
        
    def forward(self, x):
        x_emb = self.cate_embedding(x) # (n_batch, emb_dim)
        x_emb = self.drop(x_emb)
        x_rep = self.fc(x_emb) # (n_batch, out_dim)
        x_rep = F.relu(x_rep)
        return x_rep

In [19]:
class ConvNewsEncoder(nn.Module):
    def __init__(self, word_emb, cate_emb, 
                 filter_num, window_size, query_dim, dropout, args
                ):
        super().__init__()
        self.args = args
        if 'use_relu' not in args:
            args['use_relu'] = False
        self.word_embedding = nn.Embedding.from_pretrained(word_emb)
        self.cate_embedding = nn.Embedding.from_pretrained(cate_emb)
        self.word_emb_dim = word_emb.shape[1]
        self.cate_emb_dim = cate_emb.shape[1]
        self.title_encoder = TextEncoder(self.word_embedding, self.word_emb_dim, 
                                 filter_num, window_size, query_dim, dropout, args['use_relu'])
        if args['use_body']:
            self.body_encoder = TextEncoder(self.word_embedding, self.word_emb_dim, 
                                     filter_num, window_size, query_dim, dropout, args['use_relu'])
            self.attn = AttentionPooling(filter_num, query_dim)
        if args['use_cate']:
            self.cate_encoder = CateEncoder(self.cate_embedding, self.cate_emb_dim, filter_num, dropout)
            self.subcate_encoder = CateEncoder(self.cate_embedding, self.cate_emb_dim, filter_num, dropout)
            self.attn = AttentionPooling(filter_num, query_dim)
    
    def forward(self, news):
        title, body, cate, subcate = news[:, :max_title], news[:, max_title: -2], news[:, -2], news[:, -1]
        
        r_t = self.title_encoder(title) # (n_news, emb_dim)
        
        if self.args['use_body'] and args['use_cate']:
            r_b = self.body_encoder(body) # (n_news, emb_dim)
            r_c = self.cate_encoder(cate) # (n_news, emb_dim)
            r_sc = self.subcate_encoder(subcate) # (n_news, emb_dim)
            r = torch.stack((r_t, r_b, r_c, r_sc), dim = 1) # (n_news, 4, emb_dim)
            r = self.attn(r) # (n_news, n_filter)
        elif self.args['use_body']:
            r_b = self.body_encoder(body) # (n_news, emb_dim)
            r = torch.stack((r_t, r_b), dim = 1) # (n_news, 4, emb_dim)
            r = self.attn(r) # (n_news, n_filter)
        elif self.args['use_cate']:
            r_c = self.cate_encoder(cate) # (n_news, emb_dim)
            r_sc = self.subcate_encoder(subcate) # (n_news, emb_dim)
            r = torch.stack((r_t, r_c, r_sc), dim = 1) # (n_news, 4, emb_dim)
            r = self.attn(r) # (n_news, n_filter)
        else:
            r = r_t
        return r # (n_news, n_filter)

class UserEncoder(nn.Module):
    def __init__(self, emb_dim, query_dim):
        super().__init__()
        self.attn = AttentionPooling(emb_dim, query_dim)
    
    def forward(self, h, mask = None): 
        u = self.attn(h, mask)
        return u

In [255]:
class NAML(nn.Module):
    def __init__(self, word_emb, cate_emb, args):
        super().__init__()
        filter_num, window_size, query_dim, dropout = 400, 3, 200, 0.2
        self.news_encoder = ConvNewsEncoder(word_emb, cate_emb, filter_num, window_size, query_dim, dropout, args)
        self.user_encoder = UserEncoder(filter_num, query_dim)
    
    def forward(self, hist, samp):
        n_batch, n_news, n_sequence = hist.shape
        n_samp = samp.shape[1] # k + 1
        
        hist = hist.reshape(n_batch * n_news, n_sequence)
        h = self.news_encoder(hist) # (n_batch*n_news, n_filter)
        h = h.reshape(n_batch, n_news, -1)  # (n_batch, n_news, n_filter)
        u = self.user_encoder(h) # (n_batch, n_filter)
        
        samp = samp.reshape(n_batch * n_samp, n_sequence)
        r = self.news_encoder(samp) # (n_batch*(k+1), n_filter)
        r = r.reshape(n_batch, n_samp, -1) # (n_batch, k + 1, n_filter)
        
        y = torch.bmm(r, u.unsqueeze(2)) # (n_batch, K + 1, 1)
        return y.squeeze(2)

## NRMS

In [21]:
# def clones(module, n):
#     return nn.ModuleList([deepcopy(module)] * n)

# class SelfAttention(nn.Module): # word_level
#     def __init__(self, emb_dim, query_dim, output_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(emb_dim, query_dim)
#         self.fc2 = nn.Linear(emb_dim, query_dim)
#         self.fc3 = nn.Linear(emb_dim, output_dim)
#     def forward(self, e): # (n_batch, n_seq, emb_dim)
#         query = self.fc1(e) # (n_batch, n_seq, query_dim)
#         key = self.fc2(e) # (n_batch, n_seq, query_dim)
#         value = self.fc3(e) # (n_batch, n_seq, output_dim)
        
#         dot = torch.matmul(query, key.transpose(-1, -2)) # (n_batch, n_seq, n_seq)
#         alpha = F.softmax(dot, dim = -1) # (n_batch, n_seq, n_seq)
#         h = torch.matmul(alpha, value) # (n_batch, n_seq, output_dim)
#         return h # (n_batch, n_seq, output_dim)

# class MultiHeadSelfAttention(nn.Module):
#     def __init__(self, h, emb_dim, query_dim, output_dim):
#         super().__init__()
#         self.h = h
#         assert(output_dim % h == 0)
#         self.layers = clones(SelfAttention(emb_dim, query_dim, output_dim // h), h)
        
#     def forward(self, x):
#         reps = [layer(x) for layer in self.layers]
#         reps = torch.cat(reps, dim = -1)
#         return reps

In [24]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super().__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask=None):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        scores = torch.exp(scores)
        if attn_mask is not None:
            scores = scores * attn_mask
        attn = scores / (torch.sum(scores, dim=-1, keepdim=True)  + 1e-8)
        
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k, d_v):
        super().__init__()
        self.d_model = d_model # 300
        self.n_heads = n_heads # 20
        self.d_k = d_k # 20
        self.d_v = d_v # 20
        
        self.W_Q = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_K = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_V = nn.Linear(d_model, d_v * n_heads) # 300, 400
        
        self._initialize_weights()
                
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                
    def forward(self, Q, K, V, attn_mask=None):
        residual, batch_size = Q, Q.size(0)
        
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)
        
        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(1).expand(batch_size, max_len, max_len) 
            attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) 
        
        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask) 
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) 
        return context # (n_batch, n_seq, emb_dim)

In [25]:
class AttnNewsEncoder(nn.Module):
    def __init__(self, word_emb, cate_emb, n_head, news_dim, query_dim):
        super().__init__()
        self.args = args
        self.word_embedding = nn.Embedding.from_pretrained(word_emb)
        emb_dim = word_emb.shape[1]
        # self.self_attn = MultiHeadSelfAttention(n_head, emb_dim, query_dim, news_dim)
        self.self_attn = MultiHeadSelfAttention(emb_dim, n_head, 16, 16)
        self.addi_attn = AttentionPooling(news_dim, query_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, news):
        title, body, cate, subcate = news[:, :max_title], news[:, max_title: -2], news[:, -2], news[:, -1]
        
        t_rep = self.word_embedding(title) # (n_batch, n_seq, emb_dim)
        t_rep = self.dropout(t_rep)
        t_rep = self.self_attn(t_rep, t_rep, t_rep) # (n_batch, n_seq, 256)
        t_rep = self.addi_attn(t_rep) # (n_batch, 256)
        
        return t_rep # (n_news, 256)

class AttnUserEncoder(nn.Module):
    def __init__(self, n_head, news_dim, query_dim):
        super().__init__()
        self.self_attn = MultiHeadSelfAttention(news_dim, n_head, 16, 16)
        self.addi_attn = AttentionPooling(news_dim, query_dim)
    
    def forward(self, h): # (n_batch, n_news, 256)
        u = self.self_attn(h, h, h) # (n_batch, n_news, 256)
        u = self.addi_attn(u) # (n_batch, 256)
        return u

In [54]:
class NRMS(nn.Module):
    def __init__(self, word_emb, cate_emb, args):
        super().__init__()
        n_head, query_dim, news_dim = 16, 200, 256
        self.news_encoder = AttnNewsEncoder(word_emb, cate_emb, n_head, news_dim, query_dim)
        self.user_encoder = AttnUserEncoder(n_head, news_dim, query_dim)
    
    def forward(self, hist, samp):
        n_batch, n_news, n_sequence = hist.shape
        n_samp = samp.shape[1] # k + 1
        
        hist = hist.reshape(n_batch * n_news, n_sequence)
        h = self.news_encoder(hist) # (n_batch*n_news, n_filter)
        h = h.reshape(n_batch, n_news, -1)  # (n_batch, n_news, n_filter)
        u = self.user_encoder(h) # (n_batch, n_filter)
        
        samp = samp.reshape(n_batch * n_samp, n_sequence)
        r = self.news_encoder(samp) # (n_batch*(k+1), n_filter)
        r = r.reshape(n_batch, n_samp, -1) # (n_batch, k + 1, n_filter)
        
        y = torch.bmm(r, u.unsqueeze(2)) # (n_batch, K + 1, 1)
        return y.squeeze(2)

## LSTUR

In [26]:
class GruUserEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, 1, batch_first = True)

    def forward(self, h): # (n_batch, n_news, news_dim)
        h0 = torch.randn((1, h.shape[0], self.hidden_size), device = 'cuda')
        output, hn = self.gru(h, h0)
        return hn.squeeze(0) # (n_batch, news_dim)
        

In [27]:
class Model(nn.Module):
    def __init__(self, args, word_emb, cate_emb):
        super().__init__()
        if args['model'] == 'NAML':
            filter_num, window_size, query_dim, dropout = 400, 3, 200, 0.2
            args['use_relu'] = False
            self.news_encoder = ConvNewsEncoder(word_emb, cate_emb, filter_num, window_size, query_dim, dropout, args)
            self.user_encoder = UserEncoder(filter_num, query_dim)
        if args['model'] == 'NRMS':
            n_head, query_dim, news_dim = 16, 200, 256
            self.news_encoder = AttnNewsEncoder(word_emb, cate_emb, n_head, news_dim, query_dim)
            self.user_encoder = AttnUserEncoder(n_head, news_dim, query_dim)
        if args['model'] == 'LSTUR':
            filter_num, window_size, query_dim, dropout = 300, 3, 200, 0.2
            args['use_relu'] = True
            self.news_encoder = ConvNewsEncoder(word_emb, cate_emb, filter_num, window_size, query_dim, dropout, args)
            self.user_encoder = GruUserEncoder(filter_num, filter_num)
        if args['model'] == 'CAUM':
            filter_num, window_size, query_dim, dropout = 400, 3, 200, 0.2
            args['use_relu'] = False
            self.news_encoder = ConvNewsEncoder(word_emb, cate_emb, filter_num, window_size, query_dim, dropout, args)
            self.user_encoder = UserEncoder(filter_num, query_dim)
    
    def forward(self, hist, samp, hist_ents = None, samp_ents = None):
        n_batch, n_news, n_sequence = hist.shape
        n_samp = samp.shape[1] # k + 1
        
        hist = hist.reshape(n_batch * n_news, n_sequence)
        h = self.news_encoder(hist) # (n_batch*n_news, n_filter)
        h = h.reshape(n_batch, n_news, -1)  # (n_batch, n_news, n_filter)
        u = self.user_encoder(h) # (n_batch, n_filter)
        
        samp = samp.reshape(n_batch * n_samp, n_sequence)
        r = self.news_encoder(samp) # (n_batch*(k+1), n_filter)
        r = r.reshape(n_batch, n_samp, -1) # (n_batch, k + 1, n_filter)
        y = torch.bmm(r, u.unsqueeze(2)) # (n_batch, K + 1, 1)
        return y.squeeze(2)

## CAUM

In [28]:
caum_train_dataset = TrainDataset(train_datas, train_users, train_labels, news_info, train_user_hist, 16, news_ents)

In [29]:
class EntNewsEncoder(nn.Module):
    def __init__(self, word_emb, cate_emb, ent_emb,
                n_head, query_dim, news_dim
                ):
        super().__init__()
        self.args = args
        self.word_embedding = nn.Embedding.from_pretrained(word_emb)
        self.cate_embedding = nn.Embedding.from_pretrained(cate_emb)
        self.ent_embedding = nn.Embedding.from_pretrained(ent_emb)
        word_dim = word_emb.shape[1]
        cate_dim = cate_emb.shape[1]
        ent_dim = ent_emb.shape[1]
        
        self.word_transformer = MultiHeadSelfAttention(word_dim, n_head, 16, 16)
        self.word_attn = AttentionPooling(news_dim, query_dim)
        self.ent_transformer = MultiHeadSelfAttention(ent_dim, n_head, 16, 16)
        self.ent_attn = AttentionPooling(news_dim, query_dim)
        self.cate_encoder = CateEncoder(self.cate_embedding, cate_dim, news_dim)
        self.subcate_encoder = CateEncoder(self.cate_embedding, cate_dim, news_dim)
        self.aggr = nn.Linear(news_dim * 4, news_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, news, ents):
        title, body, cate, subcate = news[:, :max_title], news[:, max_title: -2], news[:, -2], news[:, -1]
        # print(title.shape, cate.shape, ents.shape)

        t_rep = self.word_embedding(title) # (n_batch, n_seq, emb_dim)
        t_rep = self.dropout(t_rep)
        t_rep = self.word_transformer(t_rep, t_rep, t_rep) # (n_batch, n_seq, 256)
        t_rep = self.word_attn(t_rep) # (n_batch, 256)
        
        c_rep = self.cate_encoder(cate)
        sc_rep = self.cate_encoder(subcate)
        
        e_rep = self.ent_embedding(ents) # (n_batch, n_ent, emb_dim)
        e_rep = self.dropout(e_rep)
        e_rep = self.ent_transformer(e_rep, e_rep, e_rep) # (n_batch, n_ent, 256)
        e_rep = self.ent_attn(e_rep) # (n_batch, 256)
        
        # print(t_rep.shape, e_rep.shape, c_rep.shape)
        r = torch.cat((t_rep, c_rep, sc_rep, e_rep), dim = 1)
        r = self.aggr(r)
        return r # (n_batch, 256)    

In [30]:
class CandiAwareCNN(nn.Module):
    def __init__(self, news_dim, filter_num, window_size):
        super().__init__()
        self.cnn = nn.Conv1d(news_dim, filter_num, window_size, padding = window_size // 2)
        self.candi_fc = nn.Linear(news_dim, filter_num)
    def forward(self, c, n):
        c_rep = self.cnn(c.transpose(2, 1)).transpose(2, 1) # (n_batch, n_seq, filter_num)
        n_rep = self.candi_fc(n) # (n_batch, 1, filter_num)
        s = c_rep + n_rep.expand(c_rep.shape) # (n_batch, n_seq, filter_num)
        return s

In [31]:
class CandiAttentionPooling(nn.Module):
    def __init__(self, emb_dim, query_dim):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim * 2, query_dim)
        self.fc2 = nn.Linear(query_dim, 1)
        
    def forward(self, m, n):
        a = torch.cat((m, n.expand(m.shape)), dim = 2) # (n_batch, n_seq, emb_dim * 2)
        a = self.fc2(torch.tanh(self.fc1(a))) # (n_batch, n_seq, 1)
        alpha = F.softmax(a, dim = 1) # (n_batch, n_seq, 1)
        r = torch.bmm(alpha.transpose(-1, -2), m).squeeze(1) # (n_batch, emb_dim)
        return r

In [32]:
class CandiUserEncoder(nn.Module):
    def __init__(self, news_dim, query_dim, window_size):
        super().__init__()
        hidden_dim = 256
        self.self_attn = MultiHeadSelfAttention(news_dim, 16, 16, 16)
        self.candi_cnn = CandiAwareCNN(news_dim, news_dim, window_size)
        self.aggr = nn.Linear(news_dim + news_dim, news_dim)
        self.candi_attn = CandiAttentionPooling(news_dim, query_dim)
        
        self.fc1 = nn.Linear(news_dim + news_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, news_dim)
        self.attn = AttentionPooling(news_dim, hidden_dim)

    def forward(self, c, n): # (n_batch, n_news, news_dim)
        '''
        c: click, (n_batch, n_seq, news_dim)
        n: candidate, (n_batch, 1, news_dim)
        '''
        n = n.unsqueeze(-2)
        s = self.self_attn(c + n, c + n, c + n) # (n_batch, n_seq, output_dim)
        l = self.candi_cnn(c, n) # (n_batch, n_seq, filter_num)
        m = self.aggr(torch.cat((s, l), dim = -1)) # (n_batch, n_seq, news_dim)
        u = self.candi_attn(m, n)
        return u # (n_batch, news_dim)

In [52]:
class CAUM(nn.Module):
    def __init__(self, word_emb, cate_emb, ent_emb, args):
        super().__init__()
        news_dim, query_dim, window_size, n_head = 256, 200, 3, 16
        self.news_encoder = EntNewsEncoder(word_emb, cate_emb, ent_emb,
                                n_head, query_dim, news_dim)
        self.user_encoder = CandiUserEncoder(news_dim, query_dim, window_size)

    def forward(self, hist, samp, samp_ents, hist_ents): # (n_batch, n_news, news_dim)
        n_batch, n_news, n_sequence = hist.shape
        n_samp = samp.shape[1] # k + 1
        n_ent = samp_ents.shape[2]
        
        hist = hist.reshape(n_batch * n_news, n_sequence)
        hist_ents = hist_ents.reshape(n_batch * n_news, n_ent)
        h = self.news_encoder(hist, hist_ents) # (n_batch*n_news, news_dim)
        h = h.reshape(n_batch, n_news, -1)  # (n_batch, n_news, news_dim)
        
        samp = samp.reshape(n_batch * n_samp, n_sequence)
        samp_ents = samp_ents.reshape(n_batch * n_samp, n_ent)
        r = self.news_encoder(samp, samp_ents) # (n_batch*(k+1), news_dim)
        r = r.reshape(n_batch, n_samp, -1) # (n_batch, k + 1, news_dim)
        
        u = []
        for i in range(n_samp):
            u.append(self.user_encoder(h, r[:, i])) 
        u = torch.stack(u, dim = 1) # (n_batch, k + 1, news_dim)
        y = torch.sum(torch.mul(u, r), dim = 2) # (n_batch, K + 1)
        return y

In [34]:
def train_caum(model, train_dataset, epochs = 4):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_losses = []
        model.train()
        for _, batch in tqdm(enumerate(train_dataset)):
            if batch[0].shape[0] == 0:
                break
            sample = torch.tensor(batch[0], dtype = torch.long, device = device)
            history = torch.tensor(batch[1], dtype = torch.long, device = device)
            correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
            samp_ents = torch.tensor(batch[3], dtype = torch.long, device = device)
            user_ents = torch.tensor(batch[4], dtype = torch.long, device = device)
            optimizer.zero_grad()
            output = model(history, sample, samp_ents, user_ents)
            loss = entrophy(output, correct)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print('[epoch {:d}] train_loss: {:.4f}'.format(epoch + 1, np.average(train_losses)))

In [35]:
def encode_caum_all_news(news_info, news_ents, news_encoder):
    n_news = len(news_info)
    news_rep = []
    n_batch = 32
    for i in range((len(news_info) + n_batch - 1) // n_batch):
        batch_news = torch.tensor(news_info[i * n_batch: (i + 1) * n_batch], dtype = torch.long, device = 'cuda')
        batch_ents = torch.tensor(news_ents[i * n_batch: (i + 1) * n_batch], dtype = torch.long, device = 'cuda')
        batch_rep = news_encoder(batch_news, batch_ents).detach().cpu().numpy()
        news_rep.append(batch_rep)
    news_rep = np.concatenate(news_rep, axis = 0)
    return news_rep # (n_news, n_title, n_emb)

In [84]:
def evaluate_caum(model, dev_dataset, news_info, news_ents):
    news_rep = encode_caum_all_news(news_info, news_ents, model.news_encoder)
    model.eval()
    with torch.no_grad():
        auc_scores = []
        mrr_scores = []
        ndcg5_scores = []
        ndcg10_scores = []
        for i, batch in tqdm(enumerate(dev_dataset)):
            if len(batch[0]) == 0:
                break
            user_hist_rep = torch.tensor(news_rep[batch[1]], device = 'cuda') # (n_batch, n_hist, news_dim)
            for j in range(len(batch[0])): # n_batch
                imp_rep = torch.tensor(news_rep[batch[0][j]], device = 'cuda') # (n_imp, news_dim)
                user = model.user_encoder(user_hist_rep[[j] * imp_rep.shape[0]], imp_rep)
                score = torch.sum(torch.mul(user, imp_rep), dim = -1) # (n_imp)
                predict = F.softmax(score, dim = 0).detach().cpu().numpy()
                positive = batch[2][j] # [n_imp]
                
                auc_scores.append(roc_auc_score(positive, predict))
                mrr_scores.append(mrr_score(positive, predict))
                ndcg5_scores.append(ndcg_score(positive, predict, k = 5))
                ndcg10_scores.append(ndcg_score(positive, predict, k = 10))
    print('[Test] AUC: {:4f}, MRR: {:4f}, nDCG5:{:4f}, nDCG10: {:4f}'.format(
        np.mean(auc_scores), np.mean(mrr_scores), np.mean(ndcg5_scores), np.mean(ndcg10_scores)
    ))

In [85]:
# train
def train_and_eval_caum(model, train_dataset, dev_dataset, news_info, news_ents, epochs = 4):
    optimizer = optim.Adam(model.parameters(), lr = 1e-5)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_losses = []
        valid_losses = []
        model.train()
        for _, batch in tqdm(enumerate(train_dataset)):
            if batch[0].shape[0] == 0:
                break
            sample = torch.tensor(batch[0], dtype = torch.long, device = device)
            history = torch.tensor(batch[1], dtype = torch.long, device = device)
            correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
            samp_ents = torch.tensor(batch[3], dtype = torch.long, device = device)
            user_ents = torch.tensor(batch[4], dtype = torch.long, device = device)
            optimizer.zero_grad()
            output = model(history, sample, samp_ents, user_ents)
            loss = entrophy(output, correct)
            train_losses.append(loss.item())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
        print('[epoch {:d}] train_loss: {:.4f}'.format(epoch + 1, np.average(train_losses)))
        evaluate_caum(model, dev_dataset, news_info, news_ents)

## FIM

In [ ]:
class HDCNewsEncoder(nn.Module): # Hierarchical dilated convolution
    def __init__(self, args, word_emb, news_dim, strides = [1, 2, 3]):
        super().__init__()
        self.args = args
        self.use_cate = args['use_cate']
        self.word_embedding = nn.Embedding.from_pretrained(word_emb)
        
        self.cnns = torch.ModuleList([nn.Conv1d(word_emb.shape[1], news_dim, 3, stride = i, padding = 1) for i in strides])
        self.layernorm = nn.LayerNorm(word_emb.shape[1])
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, news):
        title, body, cate, subcate = news[:, :max_title], news[:, max_title: -2], news[:, -2], news[:, -1]
        if self.use_cate:
            title = torch.cat((title, cate, subcate), dim = 1)
        
        t_rep = self.word_embedding(title) # (n_batch, n_seq, emb_dim)
        t_rep = self.dropout(t_rep)
        r = []
        for cnn in self.cnns:
            l = F.relu(cnn(t_rep.transpose(-1, -2)).transpose(-1, -2))
            l = self.layernorm(l) # (n_batch, n_seq, n_filter)
            r.append(l)
        
        r = torch.stack(r, dim = 1) # (n_batch, n_layer, n_seq, n_filter)
        
        return r
        

In [ ]:
class FIM(nn.Module):
    def __init__(self, word_emb, args):
        super().__init__()
        strides = [1, 2, 3]
        self.news_encoder = HDCNewsEncoder(args, word_emb, 150, strides)
        self.cnn1 = nn.Conv3d(len(strides), 32, 3, padding = 1)
        self.cnn2 = nn.Conv3d(len(strides), 16, 3, padding = 1)
        self.pool = nn.MaxPool3d((3, 3, 3), (3, 3, 3))
        self.fc = nn.Linear(48, 1)
    
    def forward(self, hist, samp):
        n_batch, n_news, n_seq = hist.shape
        n_samp = samp.shape[1] # k + 1
        
        hist = hist.reshape(n_batch * n_news, n_seq)
        h = self.news_encoder(hist) # (n_batch*n_news, n_layer, n_seq, n_filter)
        samp = samp.reshape(n_batch * n_samp, n_seq)
        c = self.news_encoder(samp) # (n_batch*(k+1), n_layer, n_seq, n_filter)
        
        n_layer, news_dim = h.shape[-3], h.shape[-1]
        h = h.reshape(n_batch, n_news, n_layer, n_seq, news_dim).transpose(1, 0) # (n_news, n_batch, n_layer, n_seq, news_dim)
        c = c.reshape(n_batch, n_samp, n_layer, n_seq, news_dim).transpose(1, 0) # (k + 1, n_batch, n_layer, n_seq, news_dim)
        
        m = torch.zeros((n_samp, n_news, n_batch, n_layer, n_seq, n_seq))
        for i in range(c.shape[0]):
            m[i] = torch.matmul(h, c[i]) / torch.sqrt(news_dim + 1e-8) # (n_layer, n_seq, n_seq)
        m = m.permute(2, 0, 3, 1) # (n_batch, n_samp, n_layer, n_news, n_seq, n_seq)
        
        q1 = self.cnn1(m) # (n_batch, n_samp, n_filter, n_news, n_seq, n_seq)
        q2 = self.cnn1(m) # (n_batch, n_samp, n_filter, n_news, n_seq, n_seq)
        q1 = self.pool(q1).squeeze()
        q2 = self.pool(q2).squeeze()
        s = torch.cat(q1, q2, dim = -1) # (n_batch, n_samp, q1 + q2)
        
        y = self.fc(s).squeeze() # (n_batch, n_samp)
        return y
    
    def predict(self, h, c): # (n_news, n_layer, n_seq, n_filter)
        n_news, n_layer, n_seq, news_dim = h.shape
        n_samp = c.shape[1] # k + 1
        
        m = torch.zeros((n_samp, n_news, n_layer, n_seq, n_seq))
        for i in range(c.shape[0]):
            m[i] = torch.matmul(h, c[i]) / torch.sqrt(news_dim + 1e-8) # (n_layer, n_seq, n_seq)
        m = m.transpose(2, 1) # (n_samp, n_layer, n_news, n_seq, n_seq)
        
        q1 = self.cnn1(m) # (n_samp, n_filter, n_news, n_seq, n_seq)
        q2 = self.cnn1(m) # (n_samp, n_filter, n_news, n_seq, n_seq)
        q1 = self.pool(q1).squeeze()
        q2 = self.pool(q2).squeeze()
        s = torch.cat(q1, q2, dim = -1) # (n_samp, q1 + q2)
        
        y = self.fc(s).squeeze() # (n_samp)
        y = F.softmax(y, dim = -1)
        return y

In [87]:
def evaluate_fim(model, dev_dataset, news_info, dev_users, dev_user_hist):
    news_rep = encode_all_news(news_info, model.news_encoder) # (65238, 400)
    
    model.eval()
    with torch.no_grad():
        auc_scores = []
        mrr_scores = []
        ndcg5_scores = []
        ndcg10_scores = []
        for i, batch in tqdm(enumerate(dev_dataset)):
            if len(batch[0]) == 0:
                break
            for j in range(len(batch[0])):
                samp = news_rep[batch[0][j]] # (n_imp, emb_dim)
                hist = news_rep[batch[1][j]] # (n_hist, emb_dim)
                positive = batch[2][j] # [n_imp]

                score = model.predict(hist, samp) # (n_imp)

                auc_scores.append(roc_auc_score(positive, predict))
                mrr_scores.append(mrr_score(positive, predict))
                ndcg5_scores.append(ndcg_score(positive, predict, k = 5))
                ndcg10_scores.append(ndcg_score(positive, predict, k = 10))
    print('[Test] AUC: {:4f}, MRR: {:4f}, nDCG5:{:4f}, nDCG10: {:4f}'.format(
        np.mean(auc_scores), np.mean(mrr_scores), np.mean(ndcg5_scores), np.mean(ndcg10_scores)
    ))

In [ ]:
args = {'model': 'FIM', 
        'use_cate': True}
print(args)
model = FIM(word_emb, args).to(device)
train(model, train_dataset)
# train_and_eval_caum(model, train_dataset, dev_dataset, news_info, news_ents)

# Test

In [37]:
def train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, epochs = 4):
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)
    entrophy = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        train_losses = []
        valid_losses = []
        model.train()
        for _, batch in tqdm(enumerate(train_dataset)):
            if batch[0].shape[0] == 0:
                break
            # torch.Size([16, 5, 30]) torch.Size([16, 50, 30]) torch.Size([16])
            sample = torch.tensor(batch[0], dtype = torch.long, device = device)
            history = torch.tensor(batch[1], dtype = torch.long, device = device)
            correct = torch.argmax(torch.tensor(batch[2], dtype = torch.long, device = device), dim = 1)
            optimizer.zero_grad()
            output = model(history, sample)
            loss = entrophy(output, correct)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print('[epoch {:d}] train_loss: {:.4f}'.format(epoch + 1, np.average(train_losses)))
        evaluate(model, dev_dataset, news_info, dev_users, dev_user_hist)

In [146]:
args = {'model': 'NAML',
        'use_body': False,
        'use_cate': False,}
print(args)
model = Model(args, word_emb, cate_emb).to(device)
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, epochs = 10)

{'model': 'NAML', 'use_body': False, 'use_cate': False}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.4359


0it [00:00, ?it/s]

[Test] AUC: 0.620449, MRR: 0.280897, nDCG5:0.307829, nDCG10: 0.374122


...

[Test] AUC: 0.616360, MRR: 0.289379, nDCG5:0.311047, nDCG10: 0.377310


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.3296


0it [00:00, ?it/s]

[Test] AUC: 0.624136, MRR: 0.298484, nDCG5:0.321698, nDCG10: 0.386311


In [149]:
args = {'model': 'NAML',
        'use_body': False,
        'use_cate': True,}
print(args)
model = Model(args, word_emb, cate_emb).to(device)
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, epochs = 10)

{'model': 'NAML', 'use_body': False, 'use_cate': True}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.4134


0it [00:00, ?it/s]

[Test] AUC: 0.638183, MRR: 0.289917, nDCG5:0.318142, nDCG10: 0.383749


...

[Test] AUC: 0.649645, MRR: 0.301144, nDCG5:0.334067, nDCG10: 0.396529


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.3096


0it [00:00, ?it/s]

[Test] AUC: 0.632101, MRR: 0.296386, nDCG5:0.321526, nDCG10: 0.386385


In [150]:
args = {'model': 'NRMS',
        'use_body': False,
        'use_cate': False,}
print(args)
model = Model(args, word_emb, cate_emb).to(device)
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, epochs = 10)

{'model': 'NRMS', 'use_body': False, 'use_cate': False}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.4435


0it [00:00, ?it/s]

[Test] AUC: 0.624454, MRR: 0.282437, nDCG5:0.304589, nDCG10: 0.372709


...

[Test] AUC: 0.646973, MRR: 0.295871, nDCG5:0.323469, nDCG10: 0.390691


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.2538


0it [00:00, ?it/s]

[Test] AUC: 0.648600, MRR: 0.300377, nDCG5:0.330926, nDCG10: 0.396182


In [153]:
args = {'model': 'LSTUR',
        'use_body': False,
        'use_cate': False,}
print(args)
model = Model(args, word_emb, cate_emb).to(device)
train_and_eval(model, train_dataset, dev_dataset, news_info, dev_users, dev_user_hist, epochs = 10)

{'model': 'LSTUR', 'use_body': False, 'use_cate': False}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.4210


0it [00:00, ?it/s]

[Test] AUC: 0.630507, MRR: 0.286492, nDCG5:0.312122, nDCG10: 0.377909


...

[Test] AUC: 0.653731, MRR: 0.302604, nDCG5:0.330371, nDCG10: 0.395960


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.2512


0it [00:00, ?it/s]

[Test] AUC: 0.660234, MRR: 0.306295, nDCG5:0.338878, nDCG10: 0.401522


In [60]:
args = {'model': 'CAUM', 
        'use_body': False,
        'use_cate': False,
        'use_ent': True}
print(args)

model = CAUM(word_emb, cate_emb, ent_emb, args).to('cuda')
train_and_eval_caum(model, caum_train_dataset, dev_dataset, news_info, news_ents, 10)

{'model': 'CAUM', 'use_body': False, 'use_cate': False, 'use_ent': True}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.4240


0it [00:00, ?it/s]

[Test] AUC: 0.638464, MRR: 3.703349, nDCG5:11.973679, nDCG10: 33.167233


...

[Test] AUC: 0.666925, MRR: 3.703349, nDCG5:11.973679, nDCG10: 33.167233


0it [00:00, ?it/s]

[epoch 8] train_loss: nan


0it [00:00, ?it/s]

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [86]:
args = {'model': 'CAUM', 
        'use_body': False,
        'use_cate': False,
        'use_ent': True}
print(args)

model = CAUM(word_emb, cate_emb, ent_emb, args).to('cuda')
train_and_eval_caum(model, caum_train_dataset, dev_dataset, news_info, news_ents, 10)

{'model': 'CAUM', 'use_body': False, 'use_cate': False, 'use_ent': True}


0it [00:00, ?it/s]

[epoch 1] train_loss: 1.5160


0it [00:00, ?it/s]

[Test] AUC: 0.578083, MRR: 0.264291, nDCG5:0.282634, nDCG10: 0.347947


...

[Test] AUC: 0.639981, MRR: 0.297714, nDCG5:0.328512, nDCG10: 0.393647


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.3444


0it [00:00, ?it/s]

[Test] AUC: 0.642799, MRR: 0.297690, nDCG5:0.328815, nDCG10: 0.394010


In [88]:
train_and_eval_caum(model, caum_train_dataset, dev_dataset, news_info, news_ents, 10)

0it [00:00, ?it/s]

[epoch 1] train_loss: 1.3399


0it [00:00, ?it/s]

[Test] AUC: 0.650674, MRR: 0.307282, nDCG5:0.337480, nDCG10: 0.401941


0it [00:00, ?it/s]

...

[Test] AUC: 0.661723, MRR: 0.313522, nDCG5:0.345583, nDCG10: 0.409894


0it [00:00, ?it/s]

[epoch 10] train_loss: 1.3055


0it [00:00, ?it/s]

[Test] AUC: 0.665166, MRR: 0.314975, nDCG5:0.347614, nDCG10: 0.411923
